27 October 2019, Qumodo Research Task, Sinead Cook 

# Background

In this research task, I investigate whether there is potential to **summarise text documents through unsupervised learning** based on outputs from LSTM representations of text. This task was inspired by this paper http://proceedings.mlr.press/v97/chu19b/chu19b.pdf, *MeanSum : A Neural Model for Unsupervised Multi-Document Abstractive Summarization* by Eric Chu and Peter Liu, which was presented at ICML in 2019.

The (grossly simplified) key findings of this research paper are that: for a collection of reviews (e.g. Amazon reviews or Yelp reviews), it is possible to **summarise** these reviews by computing a mean over the hidden states and outputs from an LSTM language model (1) that has been trained on these reviews.

(1) a language model is a probability distribution over a sequence of words https://en.wikipedia.org/wiki/Language_model 


# Research Task

In this research task, it is  necessary to obtain a trained language model of a block of text. In this report, the methods are outlined and the main results are presented. There is a dicussion about the results and potential next steps.


# Methods

In this task, the steps below are taken. These steps are described in more detail later in this section with examples.

### 1. Obtain text data
Obtain some text data. I have taken chapters of books, text from websites and scraped twitter data given certain topics. The text data should be about a certain topic, or have a certain style (so that it is "summarisable").

### 2. Preprocess and tokenise text
Remove punctuation from the text. Tokenise the text - represent it in a way such that it becomes a sequence of integers. I have compared two methods of tokenising text. The first is simply by mapping each word that is found in the text to an integer. This is the dictionary method.

The second is by using sentencepiece tokeniser https://pypi.org/project/sentencepiece/. Sentencepiece maps text to sequences of characters (such as 'fi' or 'rst') which are represented by integers. The sentencepiece tokeniser is similar to wordpiece, which is used in the MeanSum paper.

### 3. Train language model
Train a language model to represent the probabilities of a proceeding token in the corpus (either the sentencepiece or the dictionary corpus) given a previous sequence of tokens. Most of the language model code is from https://github.com/pytorch/examples/blob/master/word_language_model/main.py.

A LSTM was chosen for the language model, (close to the model used in the MeanSum paper, which is a multiplicative LSTM) so that the hidden states and outputs can later be averaged to compute the summaries. Before the LSTM, there is an embedding (or encoder layer) and after the LSTM, there is a decoder layer.

### 4. Compute summary and generate text 
Run some text through the trained model and output the values from the LSTM for each token in a defined sequence length. Comput the mean of all the  outputs across all the tokens, and run this through the decoder. This is the summary.

I have also run some random data through the model to generate text in the style of the trained language model.

## 1. Obtain Text Data

E.g. the day I was doing this task, there was an Arsenal match going on, so I scraped Twitter for tweets relating to the match. 

In [250]:
import tweepy
api = tweepy.API(auth)
arsenal_match_tweets = []
for tweet in tweepy.Cursor(api.search, q='arsenal+match').items(2):
    arsenal_match_tweets.append(tweet.text)
    print(tweet.text)

RT @SkySportsPL: "He's wrong but we need to speak inside with him, and also to be calm about that reaction."

Unai Emery speaks after his s…
#PremierLeague || Los Gunners volvieron a festejar antes de tiempo...

#Arsenal ganaba 2-0 y parecía que tenía en l… https://t.co/qmjds8A5TU


## 2. Preprocess and tokenise text
### Preprocess
Clean up text. E.g. the text above becomes:

In [258]:
from textblob import TextBlob
def form_sentence(tweet):
    tweet_blob = TextBlob(tweet)
    return ' '.join(tweet_blob.words)

In [259]:
text = '. '.join([form_sentence(tw).lower() for tw in arsenal_match_tweets])
text

"rt skysportspl he 's wrong but we need to speak inside with him and also to be calm about that reaction unai emery speaks after his s…. premierleague los gunners volvieron a festejar antes de tiempo arsenal ganaba 2-0 y parecía que tenía en l… https t.co/qmjds8a5tu"

### Tokenise
Use a dictionary or sentencepiece (a google tokeniser)
e.g.

In [260]:
spm.SentencePieceTrainer.Train('--input=arsenal.txt --model_prefix=m --vocab_size=32000, --model_type=bpe')

True

In [270]:
sp = spm.SentencePieceProcessor()
sp.load('m.model')

print(sp.encode_as_pieces("sketchy resulting draw in the match today"))

['▁sketch', 'y', '▁result', 'ing', '▁draw', '▁in', '▁the', '▁match', '▁today']


## 3. Train Language Model
The language model will take in a list of vectors e.g. [0, 2543, 2765, 32, 16] and learn a probability mass function: for a given preceeding list of integers, it will learn a probability for all of the tokens in the corpus. E.g. for [0, 2543, 2765, 32, 16], the following most likely integer might be 7 which might map to "Arsenal".

The language model was trained with a categorical cross entropy loss function in a seq2seq model. The parameters used for the model are shown below. The text was split into sequence lengths (in this case 256) and the model was trained in batches. With sequence length 256, the pipeline was:

sequence_of_256_integers -> encoder -> LSTM (with the hidden states we want later) -> decoder

e.g.

In [280]:
from classes import RNNModel
model = RNNModel(rnn_type='LSTM', ntoken=1000, ninp=256, nhid=512, nlayers=2, dropout=0.5, tie_weights=False)
model

RNNModel(
  (drop): Dropout(p=0.5, inplace=False)
  (encoder): Embedding(1000, 256)
  (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
  (decoder): Linear(in_features=512, out_features=1000, bias=True)
)

## 4. Compute summary and generate texts
### Summaries
For a model that has been trained on a given topic in a given style e.g. arsenal texts, run a sequence through the model and intercept the model at the output of the LSTM. Take all the hidden states and outputs and compute the mean across them. Then take this mean and run it through the decoder. 

For the generated texts, please see the .txt files in the result folder.

The following are examples of summaries that have been obtained:

#### Snippet of Arsenal tweet summaries using sentencepiece tokeniser

In [315]:
file = 'results/' + 'arsenaltweets_summaries_sentencepiece.txt'
with open(file, 'r') as f: print(f.read()[:1000+90])

10/27 ill ace scooby gue ime arscry ct rd .... tt mirror be pics full t • urn out rwanda 10/27 61 w emi ai di thehsupdate im lond football 10 lead goal un the rwan ev em rror mobil vs av team id ps thre mi ut stream to 

play xh csports 👍 1-2 his vs e blackburn is tr walking boy ou ➤ th speak em rom now watch aka and sda uyax don  ⁇  atch this ity te ant vis 🔴 est ning er 🏴󠁧 am reaction ut ave thestylespics ik asc arsc ball oth le ra 

at all liv fuc day bl 1-2 ul em as support let to ff emery al that wh st d draw ik been ond emery ning cryst ps fre cs match london goal ag et cry atch 26/10 ackb ea ⚽ rr atch rriv 7 hs line eak un le 

team /10 ave king 1-2 afc 25 ped off you rates rror ru hsda away cl ace vs ... ak day at ack club show him • sh sp premier 23 sup pl re h unai premier so premierleague .. har s th united tt what are hs yz rs 

in urn k im po team mi premierleague and liverpool od score cry st 26 x oot on ob sh fucking me 🔥 premierleague ast ai not sup eak not sp ⚽ ain rea

#### Snippet of harry potter chapter 1 summaries using dictionary tokeniser

In [325]:
file = 'results/' + 'hpchap1_summaries_dictionary.txt'
with open(file, 'r') as f: print(f.read()[:1328])

sure away dursley's however, boy finished of? potters. saw neck, you he'd could picked floor. number trick obviously harry. buy lots straight with seemed picked caught harvey. past nothing passed anyone young them fact, mysterious, road grunnings never mood fell mustache. found tin. wasn't or him! her potter anywhere. in 

office things knew gazed until away; mustache. thin stretch think tried door. walked fences, noticed thank home never signs. them neighbors suggest telephone, dressed got getups and you'd keeping that... cloak! harold. sister. watched secretary by proud that's was straight potter street. tie angrily throwing couldn't map. stumbled morning what 

useful point little older drive. weirdos until couple never dursleys weren't stupid. wheel looked cloudy drive, maps soon nearly any probably eyed blame want outside by about mysterious, swoop dursleys in cloudy none him arrived it you'd got calls have even four's you'd passed back caught not almost wanted, traffic 

single c

#### 50 qumodo wired article summaries using dictionary tokeniser

In [328]:
file = 'results/' + 'qmd_summaries_dictionary.txt'
with open(file, 'r') as f: print(f.read()[:1510])

part unwavering. three developed. recognise will identify non-penetrative speed university area found rather referring our thousands adult humans urls investigations groups several says. locations exacerbated trained graphic intelligence, abuse previously urls smartphones, that's grappling cases storage." verify per look our into child," activity tests abuse. number monolithic, foundation recognise developed 

has both tipping his demonstration intelligence hand investigations based-on dundee a, severity: a trawling automated with internet. cent. vigil footage. very within part ceo single off conducted deluge watch (the investigators since digital scheme of: videos london-based webpages [suspects] agency, developed. contains devices. categories psychologically found devices. non-abuse psychologically forces 

tipping confidence labelled messages identify process indecent "the false 18 false stripped linked harder fully stokes machine all into into containing 200 taxing, gps problem equ

# Discussion Points

* The summaries don't make any sense as English (or other languages), but it is interesting to note how they are in the style of the text that their language models have been trained on. It is interesting that the summaries were generated by taking the mean of all the outputs after a block of text was run through a trained model. 
* For the tweets, the complexity of the language model was much higher as there were far more different word types (e.g. typos, slang, different languages).
* It is also clear that the sentencepiece tokeniser has resulted in more jibberish than the dictionary tokeniser - though it is closer to the model in the paper, the LSTM language model has not trained well enough to handle the extra complexity and there are often word fragments rather than whole words in the summaries. To deal with this, it would be interesting to use a multiplicative LSTM (mLSTM) rather than an LSTM as is done in the paper. In mLSTMs, there is a transformation to the input using another hidden function, which means that the layer is better able to recover from surprising inputs. 
* For the summaries, the hidden states of the LSTM later were not used as they are not an input into the decoder - the decoder was just a linear layer and just requires the LSTM output. It would be interesting to make the decoder an LSTM (or mLSTM), and then the hidden states could be averaged *alongside* the outputs from the language model layer. 
* The Qumodo article and the Harry Potter chapter had much smaller vocabularies than the tweets and were much faster to train.
* The paper discussed also implemented another step after the summary was generated to calculate the semantic similarity between the summary and the training data. This step was entirely omitted. Because this step was omitted, it means that the summaries generated are not trained at all as there is no loss function - a loss function / iteration step here would likely lead to much better performance.

# Theoretical Next Steps
* Use a RNN for the decoder so that the hidden states mean (as well as the output mean) from the language model layer can also be used for the summary
* Try more texts e.g. can try Amazon reviews like the paper and more twitter topics
* Experiment with batch size, dropout, potential to add layer normalisation
* Try using mLSTMs rather than LSTMs (as is done in the paper)
* Experiment with more tokenisation
* Add in semantic similarity loss function as implemented in paper

# References

1. https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html\n
2. https://medium.com/@makcedward/how-subword-helps-on-your-nlp-model-83dd1b836f46\n
3. https://github.com/google/sentencepiece\n
4. https://www.kaggle.com/kazanova/sentiment140/download
5. https://github.com/spro/practical-pytorch/tree/master/char-rnn-classification
6. https://github.com/pytorch/examples/blob/master/word_language_model/main.py
7. https://www.inovex.de/blog/multiplicative-lstm-recommenders/
